In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.mode.chained_assignment = None
import urllib.request
import urllib 
import csv
import folium
import random
from mpl_toolkits import mplot3d 
#if something says isn't installed, !pip install [library]
##probably don't need all these, just transferred them over

In [2]:
##Adds data frames for easier date useage, we got year, month and week of year
def formatDates(point_df):
    point_df["Year"] = pd.DatetimeIndex(point_df['Date']).year
    point_df["Month"] = pd.DatetimeIndex(point_df['Date']).month
    point_df["WeekOfYear"] = pd.DatetimeIndex(point_df['Date']).weekofyear
    print("Data collection date range of point_1 ", point_df["Date"].min(), point_df["Date"].max())
    return point_df

In [3]:
#this might take a while
water_quality_final = pd.read_csv('Water_FINAL.csv', low_memory= False)
water_quality_final.shape
water_quality_final.drop(columns = {"Unnamed: 0", "Unnamed: 0.1", "Agency", "BiasPC", "CloudCover", "Comments"}, inplace = True )
#list_of_columns = list(water_quality_final.columns)
#formate the dates and such
water_quality_final['Date'] = pd.to_datetime(water_quality_final.Date)

water_quality_final["TierLevel"]= water_quality_final["TierLevel"].replace({"T3": 3, "p2": 2,"p1": 1, "3.0": 3, "2.0": 2,"1.0": 1})

water_quality_final["TierLevel"] = water_quality_final["TierLevel"].astype(int)
water_quality_final["HUCNAME_"] = water_quality_final["HUCNAME_"].str.lower()

water_temp = water_quality_final[water_quality_final["Unit"] == "DEG C"]
water_temp = water_temp["MeasureValue"].where(water_quality_final["TierLevel"] == 3)

water_quality_final = formatDates(water_quality_final)
#water temp has all the temp data 


Data collection date range of point_1  1992-11-01 00:00:00 2020-07-08 00:00:00


In [ ]:
## Run this cell if you need the list of the different creek names with the key words 

a= list(water_quality_final["HUCNAME_"].unique())
for i in range(0,len(a)-1): a[i] = a[i].lower()
susqu_creeks  = list(filter(lambda lam_s: "susquehanna" in lam_s, a)) 
bald_eagle_creeks = list(filter(lambda lam_be: "bald eagle" in lam_be, a))
lick_run_creeks = list(filter(lambda lam_lc: "lick run" in lam_lc, a)) 
beech_creeks = list(filter(lambda lam_lb: "beech creek" in lam_lb, a)) 
codorus_creeks = list(filter(lambda lam_lcor: "codorus" in lam_lcor, a)) 
pine_creeks = list(filter(lambda lam_lp: "pine" in lam_lp, a)) 
tioga_creeks = list(filter(lambda lam_lt: "tioga river" in lam_lt, a)) 

masden_creeks = list(filter(lambda lam_lm: "masden" in lam_lm, a)) 

furnace_creeks = list(filter(lambda lam_f: "furnace" in lam_f, a)) 


In [ ]:
# This will give you the data frames for the below water sheds stored in "creeks of interest" 
#if input year is zero just returns the entire data frame
def get_HUC_temp_data(HUC_name, input_df, year)

    _creek = input_df[input_df['HUCNAME_'].str.contains(HUC_name, na=False) ]
    if(year =0):
        return _creek
    else:
        _creek= _creek[(_creek["Year"] == year) & (_creek["Parameter"]=="WTEMP")]
        return _creek


In [ ]:

creeks_of_interest = "masden run-bald eagle creek", "outlet tioga river","middle tioga river", "dee run-codorus creek",\
    "beech creek","laurel run-bald eagle creek","furnace run-pine creek"]

masden_baldeagle_creek = get_HUC_temp_data(creeks_of_interest[0], water_quality_final, 0)
outlet_tioga_river = get_HUC_temp_data(creeks_of_interest[1], water_quality_final, 0)
middle_tioga_river = get_HUC_temp_data(creeks_of_interest[2], water_quality_final, 0)

dee_run_creek = get_HUC_temp_data(creeks_of_interest[3], water_quality_final, 0)
beech_creek = get_HUC_temp_data(creeks_of_interest[4], water_quality_final, 0)
laurel_run_creek = get_HUC_temp_data(creeks_of_interest[5], water_quality_final, 0)
furnace_run_creek = get_HUC_temp_data(creeks_of_interest[6], water_quality_final, 0)


In [8]:
#takes in a year array and a data frame
def plotYear_temp(years, a_creek):
    
    op = 1
    for i in years :
        a_filter= a_creek[(a_creek["Year"] == i) & (a_creek["Parameter"]=="WTEMP")]
        y = a_filter["MeasureValue"]
        x = a_filter["WeekOfYear"]

        plt.scatter( x,y, alpha = op)
        op-=0.2

    #plt.plot(x,y ,'-o')
    plt.show()
    plt.ylabel('some numbers')
years= [2012, 2018]
#plotYear_temp(years, a)

In [14]:
### This gets the annual averages from NOAA
## gets the monthly long term max and min averages for all stations, 
## stationInventory.txt gets you all the weather stations with lat and long, select the ID of the ones you want and
## place in the 'weather_station = set()' variable
## can work on 
station_inventory = 'ftp://ftp.ncdc.noaa.gov/pub/data/normals/1981-2010/station-inventories/'
urllib.request.urlretrieve('ftp://ftp.ncdc.noaa.gov/pub/data/normals/1981-2010/products/temperature/', "stationInventory.txt")
temp_data = ["mly-tmax-normal.txt" , "mly-tmin-normal.txt"]
for i in temp_data:
    urllib.request.urlretrieve('ftp://ftp.ncdc.noaa.gov/pub/data/normals/1981-2010/products/temperature/', i)

('mly-tmin-normal.txt', <email.message.Message at 0x16d1dbeddc8>)

In [ ]:
def getCSV(infile_, outfile_, weather_stations):
    a = ""
    line = ""
    with open(infile_) as infile, open(outfile_,'w', encoding='utf-8') as outfile: 
        for line in infile: 
            x = line.split(" ")

            if (x[0] in weather_stations ):
            
                for i in range(50): 

                    try:
                        #line[i+1] = line[i+1]
                        if((x[i]!= "" )):
                            t = x[i].strip()
                            if(t[-1].isalpha()):
                                
                                a =a+ t[0:-1]+","
                            else: 
                                a =a+ x[i].strip()+","

                    except IndexError:
                        t = 0
                outfile.write(a[0:-1])
                outfile.write('\n')
            a = ""

    outfile.close()
    


In [ ]:
col = ["Station" ,"JAN", "FEB", "MAR", "APR", "MAY", "JUN", "JUL", "AUG", "SEP", "OCT", "NOV", "DEC"]
def formatTempData(fileNameMin_, fileNameMax_, point_1_wtemp, file_name_):
    
    df_monthly_max = pd.read_csv("monthly_max_1.csv", error_bad_lines=False,  names = col, header = None)
    df_monthly_min = pd.read_csv("monthly_min_1.csv", error_bad_lines = False, names = col, header = None)
    ##formate the data for graphable
    df_monthly_max = df_monthly_max.T  
    df_monthly_max.columns = list(df_monthly_max.iloc[0])
    df_monthly_max.drop("Station", axis = 0, inplace = True)
    df_monthly_max["MonthNum"] = list([1,2,3,4,5,6,7,8,9,10,11,12])
    df_monthly_max = df_monthly_max.T 
##formate the data for graphable
    df_monthly_min = df_monthly_min.T  
    df_monthly_min.columns = list(df_monthly_min.iloc[0])
    df_monthly_min.drop("Station", axis = 0, inplace = True)
    df_monthly_min["MonthNum"] = list([1,2,3,4,5,6,7,8,9,10,11,12])
    df_monthly_min = df_monthly_min.T 
    plt.scatter(list(df_monthly_max.iloc[2]),np.asarray(df_monthly_max.iloc[1]/10),color='red')
    plt.scatter(list(df_monthly_min.iloc[2]),np.asarray(df_monthly_min.iloc[1]/10),color='blue')
    plt.scatter(list(point_1_wtemp["Month"]), list(point_1_wtemp["MeasureValue"]), color ="green" )
    plt.savefig("point_1_tioga.png")
    plt.show()


   
    return df_monthly_max, df_monthly_min

In [ ]:

#getCSV formates the NOAA data into a CSV, need to optimize this later since currently downloads the entire file--and we don't need it all
#formateTempData returns the data frame of monthly min and max data, as well as plots





weather_files = "mly-tmax-normal.txt" , "mly-tmin-normal.txt","monthly_max_1.csv","monthly_min_1.csv"
#weather_stations = set(["USC00364976", 'USC00364992'])
weather_stations = set(['USC00369933'])
tioga_stations = set(['USC00361838','USC00366297'])

getCSV(weather_files[0], weather_files[2], tioga_stations)
getCSV(weather_files[1], weather_files[3], tioga_stations)
monthly_min, monthly_max= formatTempData("monthly_min_1.csv" , "monthly_max_1.csv", point_1_wtemp,"Tioga.png")
monthly_min, monthly_max= formatTempData("monthly_min_1.csv" , "monthly_max_1.csv",point_1_wtemp, "point_1.png")

In [ ]:
## These functions will make different plots, distribtuion, scatter plots and 3Dplots 

def makeDistPlot(data, label, units):
    distplot = sns.distplot(data.MeasureValue)
   
    plt.xlabel('Year')
    plt.ylabel(units)
    plt.title('Distribution plot of' + label +  '\n' + units)
    plt.show()
    return distplot
def makeScatterPlot(data, label, units, ymw):
    cmap = sns.cubehelix_palette(dark=.3, light=.8, as_cmap=True)
    if(ymw == 'y'):
        plt.title("Scatter plot of mean year  of  " + label + '\n'+ units)
        plt.xlabel('Year')
        yearplot = sns.scatterplot(x = "Year", y = data["MeasureValue"].mean(), data = data)
    if(ymw == 'm'):
        plt.title("Scatter plot of monthly level of  " + label + '\n'+ units)
        plt.xlabel('Month')
        yearplot = sns.scatterplot(x = "Month", y = data["MeasureValue"], hue="Year", palette=cmap, data = data)
    if(ymw == 'w'):
        #NEEd the titles to change
        plt.title("Scatter plot of weekly level of  " + label + '\n'+ units)
        plt.xlabel('Week')
        yearplot = sns.scatterplot(x = "WeekOfYear", y = data["MeasureValue"], hue="Year",palette=cmap,data = data)
    
    plt.ylabel(units)
    plt.show()
        
    return yearplot
def make3Dfig(data, label):
    cmap = sns.cubehelix_palette(dark=.3, light=.8, as_cmap=True)
# Creating figure 
    fig = plt.figure(figsize = (10, 10)) 
    ax = plt.axes(projection ="3d") 

    # Creating plot 
    ax.scatter3D(data.Year, data.WeekOfYear, data.MeasureValue,color = "green"); 
    ax.set_xlabel("Year")
    ax.set_ylabel("Week of Year")
    ax.set_zlabel("Nutrient level")
    plt.title(label) 

    # show plot 
    plt.show() 
    return 

